In [1]:
# imports
from tensorflow.keras.models import Sequential, model_from_json, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [2]:
list_nPCs = [10, 15, 20, 25, 30, 35, 40]
takeprofit = 0.12
stoploss = 0.05

In [3]:
# (a) Load previously built datasets : we just need train sets and validation sets here
trainset_final = pd.read_csv('./Data/Trainset_final_SL{}_TP{}.csv'.format(stoploss, takeprofit))
trainset = pd.read_csv('./Data/Trainset_SL{}_TP{}.csv'.format(stoploss, takeprofit))

validation_set_final = pd.read_csv('./Data/Validationset_final_SL{}_TP{}.csv'.format(stoploss, takeprofit))
validation_set = pd.read_csv('./Data/Validationset_SL{}_TP{}.csv'.format(stoploss, takeprofit))

In [4]:
# Hotfix function
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

def make_keras_picklable():
    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))
    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

In [5]:
# (b) Build and train several models different amounts of PCs
for nPCs in list_nPCs:
    print(nPCs)
    X = trainset_final.iloc[:, :nPCs]
    y = trainset["result"]

    # Build model and train it
    classifier = Sequential()
    dropout_fraction = 0.2
    #First Hidden Layer
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=nPCs))
    classifier.add(Dropout(dropout_fraction))
    #Second, third and fourth  hidden Layers
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(dropout_fraction))
    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(dropout_fraction))
    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(dropout_fraction))

    #Output Layer
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
    #Compiling the neural network
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    #Fitting the data to the training dataset
    classifier.fit(X,y, batch_size=5, epochs=75, verbose =1)

    pk.dump(classifier, open("./Models/DL_model_{}PC_stoploss{}_takeprofit{}.pkl".format(nPCs, stoploss, takeprofit),"wb"))

===========================] - 0s 2ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 25/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 26/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0069 - accuracy: 0.9987
Epoch 27/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 28/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 29/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0370 - accuracy: 0.9893
Epoch 30/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0128 - accuracy: 0.9979
Epoch 31/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0554 - accuracy: 0.9955
Epoch 32/75
45/45 [==============================] - 0s 2ms/step - loss: 0.0202 - accuracy: 0.9955
Epoch 33/75
45/45 [==============================] - 0s 2ms/step - loss: 0.2157 - accuracy: 0.9706
Epoch 34/75
45/45 [=============

In [6]:
X_test = validation_set_final
y_test = validation_set['result']

In [7]:
# predicts = classifier.predict(X_test)

In [8]:
# classifier.evaluate(X_test, y_test)

In [9]:
accuracies, nPCs_list = [], []
for nPCs in list_nPCs:
    print(nPCs)
    with open("./Models/DL_model_{}PC_stoploss{}_takeprofit{}.pkl".format(nPCs, stoploss, takeprofit), 'rb') as f:
        clf = pk.load(f)
    # Compute predictions on testset
    preds = (clf.predict(validation_set_final.iloc[:, :nPCs]) > 0.5)*1

    # Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
    validation_set1 = validation_set[preds == 1].copy()
    accuracies.append(np.mean(preds == list(validation_set['result'])))
    nPCs_list.append(nPCs)

10
15
20
25
30
35
40


In [10]:
recap = pd.DataFrame({'nPCs' : list(nPCs_list), 'Accuracy' : (list(accuracies))})
recap.to_csv('./Results/Comparative_All_models_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit), index = False)
print(recap)

   nPCs  Accuracy
0    10     0.635
1    15     0.645
2    20     0.595
3    25     0.595
4    30     0.610
5    35     0.405
6    40     0.445


In [ ]:
# for nPCs in list_nPCs:
#     print(nPCs)
#     X = trainset_final.iloc[:, :nPCs]
#     y = trainset["result"]

#     # Build model and train it
#     classifier = Sequential()

#     dropout_fraction = 0.2

#     #First Hidden Layer
#     classifier.add(LSTM(32, activation='relu', return_sequences=True, input_shape=(X.shape[1],1)))
#     classifier.add(Dropout(dropout_fraction))

#     #Second, third and fourth  hidden Layers
#     classifier.add(Dense(32, activation='relu', return_sequences=True))
#     classifier.add(Dropout(dropout_fraction))

#     classifier.add(Dense(16, activation='relu', return_sequences=True ))
#     classifier.add(Dropout(dropout_fraction))

#     classifier.add(Dense(16, activation='relu'))
#     classifier.add(Dropout(dropout_fraction))

#     #Output Layer
#     classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#     #Compiling the neural network
#     classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#     #Fitting the data to the training dataset
#     classifier.fit(X,y, batch_size=50, epochs=75, verbose =1)

#     pk.dump(classifier, open("./Models/DL_model_{}PC_SL{}_TP{}_btc.pkl".format(nPCs, stoploss, takeprofit), 'wb'))

In [ ]:
# 10 nPCs model
X = trainset_final.iloc[:, :10]
y = trainset["result"]

# Build model and train it
classifier = Sequential()

dropout_fraction = 0.2

#First Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=10))
classifier.add(Dropout(dropout_fraction))

#Second, third and fourth  hidden Layers
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#Fitting the data to the training dataset
model = classifier.fit(X,y, batch_size=1, epochs=75, verbose =1)

In [ ]:
class_json = classifier.to_json()

file_path = Path("./Models/DL_model_10PC.json")
with open(file_path, "w") as json_file:
    json_file.write(class_json)

file_path = "./Models/DL_model_10PC.h5"
classifier.save_weights("./Models/DL_model_10PC.h5")

In [ ]:
accuracies, nPCs_list = [], []
file_path = Path("./Models/DL_model_10PC.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

file_path = Path("./Models/DL_model_10PC.h5")
loaded_model.load_weights(file_path)
# Compute predictions on testset
preds = (loaded_model.predict(validation_set_final.iloc[:, :10]) > 0.5)*1

# Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
validation_set1 = validation_set[preds == 1].copy()
accuracies.append(np.mean(preds == list(validation_set1['result'])))
nPCs_list.append(10)

In [ ]:
# 15 nPCs model
X = trainset_final.iloc[:, :15]
y = trainset["result"]

# Build model and train it
classifier = Sequential()

dropout_fraction = 0.2

#First Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=15))
classifier.add(Dropout(dropout_fraction))

#Second, third and fourth  hidden Layers
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#Fitting the data to the training dataset
classifier.fit(X,y, batch_size=1, epochs=75, verbose =1)

In [ ]:
class_json = classifier.to_json()

file_path = Path("./Models/DL_model_15PC.json")
with open(file_path, "w") as json_file:
    json_file.write(class_json)

file_path = "./Models/DL_model_15PC.h5"
classifier.save_weights("./Models/DL_model_15PC.h5")

In [ ]:
file_path = Path("./Models/DL_model_15PC.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

file_path = Path("./Models/DL_model_15PC.h5")
loaded_model.load_weights(file_path)
# Compute predictions on testset
preds = (loaded_model.predict(validation_set_final.iloc[:, :15]) > 0.5)*1

# Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
validation_set1 = validation_set[preds == 1].copy()
accuracies.append(np.mean(preds == list(validation_set1['result'])))
nPCs_list.append(15)

In [ ]:
# 20 nPCs model
X = trainset_final.iloc[:, :20]
y = trainset["result"]

# Build model and train it
classifier = Sequential()

dropout_fraction = 0.2

#First Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=20))
classifier.add(Dropout(dropout_fraction))

#Second, third and fourth  hidden Layers
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#Fitting the data to the training dataset
classifier.fit(X,y, batch_size=1, epochs=75, verbose =1)

In [ ]:
class_json = classifier.to_json()

file_path = Path("./Models/DL_model_20PC.json")
with open(file_path, "w") as json_file:
    json_file.write(class_json)

file_path = "./Models/DL_model_20PC.h5"
classifier.save_weights("./Models/DL_model_20PC.h5")

In [ ]:
file_path = Path("./Models/DL_model_20PC.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

file_path = Path("./Models/DL_model_20PC.h5")
loaded_model.load_weights(file_path)
# Compute predictions on testset
preds = (loaded_model.predict(validation_set_final.iloc[:, :20]) > 0.5)*1

# Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
validation_set1 = validation_set[preds == 1].copy()
accuracies.append(np.mean(preds == list(validation_set1['result'])))
nPCs_list.append(20)

In [ ]:
# 25 nPCs model
X = trainset_final.iloc[:, :25]
y = trainset["result"]

# Build model and train it
classifier = Sequential()

dropout_fraction = 0.2

#First Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=25))
classifier.add(Dropout(dropout_fraction))

#Second, third and fourth  hidden Layers
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#Fitting the data to the training dataset
classifier.fit(X,y, batch_size=1, epochs=75, verbose =1)

In [ ]:
class_json = classifier.to_json()

file_path = Path("./Models/DL_model_25PC.json")
with open(file_path, "w") as json_file:
    json_file.write(class_json)

file_path = "./Models/DL_model_25PC.h5"
classifier.save_weights("./Models/DL_model_25PC.h5")

In [ ]:
file_path = Path("./Models/DL_model_25PC.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

file_path = Path("./Models/DL_model_25PC.h5")
loaded_model.load_weights(file_path)
# Compute predictions on testset
preds = (loaded_model.predict(validation_set_final.iloc[:, :25]) > 0.5)*1

# Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
validation_set1 = validation_set[preds == 1].copy()
accuracies.append(np.mean(preds == list(validation_set1['result'])))
nPCs_list.append(25)

In [ ]:
# 30 nPCs model
X = trainset_final.iloc[:, :30]
y = trainset["result"]

# Build model and train it
classifier = Sequential()

dropout_fraction = 0.2

#First Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=30))
classifier.add(Dropout(dropout_fraction))

#Second, third and fourth  hidden Layers
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(dropout_fraction))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#Fitting the data to the training dataset
classifier.fit(X,y, batch_size=1, epochs=75, verbose =1)

In [ ]:
class_json = classifier.to_json()

file_path = Path("./Models/DL_model_30PC.json")
with open(file_path, "w") as json_file:
    json_file.write(class_json)

file_path = "./Models/DL_model_30PC.h5"
classifier.save_weights("./Models/DL_model_30PC.h5")

In [ ]:
file_path = Path("./Models/DL_model_30PC.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

file_path = Path("./Models/DL_model_30PC.h5")
loaded_model.load_weights(file_path)
# Compute predictions on testset
preds = (loaded_model.predict(validation_set_final.iloc[:, :30]) > 0.5)*1

# Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
validation_set1 = validation_set[preds == 1].copy()
accuracies.append(np.mean(preds == list(validation_set1['result'])))
nPCs_list.append(30)

In [ ]:
recap = pd.DataFrame({'nPCs' : list(nPCs_list), 'Accuracy' : (list(accuracies))})

In [ ]:
recap